# Demonstration of the ribbon package

When you run this for the first time, you need to install the package, which you can do by uncommenting (i.e., by removing the leading #) the line below

In [1]:
# pip install git+https://github.com/ribbon/ribbon.git

Next, we import the package and a few other ones that will be useful in this demonstration

In [2]:
import ribbon.rw  # the main class
import ribbon.visualizer as visualizer  # for visualizing the output

import snappy  # to represent the knots
import logging  # to print some info of what the code is doing

ModuleNotFoundError: No module named 'ribbon'

Here are the options we can use to configure the search

In [ ]:
max_size = 10             # max number of crossings any intermediate knot can have
max_steps =  5000         # max number of steps searched by the random walker before giving up and resetting the link
max_tries = 10000         # number of total steps before we give up completely
max_bct = 3               # we use the same variable to set the max number of allowed twists, link components, and bands to add. If you start with a knot, the max number of bands is the max number of components + 1, since each band adds a link component
log_level = logging.INFO  # controls how much information is printed by the code while searching for a band
use_checks = False        # If set to true (whcih requires sage), the code will check for slice obstructions after attaching bands, rather than keep searching more and more bands on a link that is potentially obstructed. This uses the Fox Milnor condition, which requires the Alexander Polynomial. This can be slow to compute for large knots
save_images = True        # Will use the visualizer to save the band found by the random walker

In [ ]:
random_walker = ribbon.rw.RandomWalker(links=my_links, 
                                       max_size=max_size, 
                                       max_steps=args.max_steps, 
                                       max_bct=max_bct, 
                                       logger=None, 
                                       log_level=log_level, 
                                       use_band_checks=use_checks, 
                                       save_solved_knot_images=save_images
                                      )

## Now we perform random actions that lead to random bands until we find a collection of bands that leads to the unknot, or the max number of steps has been exhausted

In [ ]:
tries = 0
while tries < max_tries:
    tries += 1
    if tries % 10000 == 0:
        logger.info("Performed {:d} steps. Currently at knot {}/{}. Solved {}.".format(tries, num_knots, len(my_links), success))
    
    # Find all valid actions
    valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()
    
    # pick a random one
    a = np.random.choice(valid_actions) if len(valid_actions) > 0 else 0
    
    # perform the action
    done, info = random_walker.step(a)
        
    # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
    if done:
        tries_per_knot += 1
        if 'unknot' in info['result']:  # found bands
            logger.error("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
            break

## Alternatively, we can weight the actions differently

In [ ]:
weights = [0.25,0.85,0.05,0.05,0.15]  # this was found to work well
weights = [float(weights[0])] * max_action_per_category + [float(weights[3]), float(weights[1]), float(weights[2])] * max_action_per_category + [float(weights[4])] * 2

In [ ]:
# the rest is essentially the same as before
tries = 0
while tries < max_tries:
    tries += 1
    if tries % 10000 == 0:
        logger.info("Performed {:d} steps. Currently at knot {}/{}. Solved {}.".format(tries, num_knots, len(my_links), success))
    
    # Find all valid actions
    valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()
    
    # pick a random valid action weighted according to the weights specified above
    ws = weights * valid_actions
    ws = ws / np.sum(ws) if np.sum(ws) != 0 else [1./(4 * max_size + 2)] * (4 * max_size + 2)
    a = np.random.choice(len(valid_actions), p=ws) if valid_actions.any() else 0
    
    # perform the action
    done, info = random_walker.step(a)
        
    # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
    if done:
        tries_per_knot += 1
        if 'unknot' in info['result']:  # found bands
            logger.error("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
            break

## Random searches can be parallelized trivially

In [ ]:
from joblib import Parallel, delayed  # for parallelization

In [ ]:
# define the function that does the search
def search_band(random_walker, max_tries=1000, weights=[1., 1., 1., 1., 1.]):
    tries = 0
    while tries < max_tries:
        tries += 1
        # Find all valid actions
        valid_actions = np.argwhere(random_walker.invalid_action_mask()).flatten()

        # pick a random valid action weighted according to the weights specified above
        ws = weights * valid_actions
        ws = ws / np.sum(ws) if np.sum(ws) != 0 else [1./(4 * max_size + 2)] * (4 * max_size + 2)
        a = np.random.choice(len(valid_actions), p=ws) if valid_actions.any() else 0

        # perform the action
        done, info = random_walker.step(a)

        # check whether the knot is done (either because the unknot was reached, or a reset was triggered)
        if done:
            tries_per_knot += 1
            if 'unknot' in info['result']:  # found bands
                logger.error("Knot is ribbon!")  # this is not an error, we just want to print this message irrespective of the logger level.
                return True
    return False

run this function multiple times in parallel

In [ ]:
n_jobs = 10  # number of parallel searches (set this <= number of cores)
backend = 'multiprocessing'  # if this does not work for you, try backend = 'loki'
rws = [ribbon.rw.RandomWalker(links=my_links, 
                                       max_size=max_size, 
                                       max_steps=args.max_steps, 
                                       max_bct=max_bct, 
                                       logger=None, 
                                       log_level=log_level, 
                                       use_band_checks=use_checks, 
                                       save_solved_knot_images=save_images
                                      )
       for _ in range(n_jobs)]  # create independent random walkers

In [ ]:
search_result = Parallel(n_jobs=n_jobs, backend=backend)([delayed(search_band)(rw) for rw in rws]
print("At least one process proved the input link is ribbon: {}". format(any(search_result)))